In [1]:
import pandas as pd
import numpy as np
import requests
import bs4 as bs
import urllib.request

## Extracting features of 2020 movies from Wikipedia

In [2]:
link = "https://en.wikipedia.org/wiki/List_of_American_films_of_2020"

In [3]:
source = urllib.request.urlopen(link).read()
soup = bs.BeautifulSoup(source,'lxml')

In [4]:
tables = soup.find_all('table',class_='wikitable sortable')

In [5]:
len(tables)

4

In [6]:
type(tables[0])

bs4.element.Tag

In [7]:
df1 = pd.read_html(str(tables[0]))[0]
df2 = pd.read_html(str(tables[1]))[0]
df3 = pd.read_html(str(tables[2]))[0]
df4 = pd.read_html(str(tables[3]).replace("'1\"\'",'"1"'))[0] # avoided "ValueError: invalid literal for int() with base 10: '1"'

<ipython-input-7-2803005693>:1: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df1 = pd.read_html(str(tables[0]))[0]
<ipython-input-7-2803005693>:2: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df2 = pd.read_html(str(tables[1]))[0]
<ipython-input-7-2803005693>:3: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df3 = pd.read_html(str(tables[2]))[0]
<ipython-input-7-2803005693>:4: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df4 = pd.read_html(str(tables[3]).replace("'1\"\'",'"1"'))[0] # avoided "V

In [9]:
df = pd.concat([df1, df2, df3, df4], ignore_index=True)

In [10]:
df

Opening  Opening.1                  Title  \
0      J A N U A R Y          3             The Grudge   
1      J A N U A R Y         10             Underwater   
2      J A N U A R Y         10            Like a Boss   
3      J A N U A R Y         10          Three Christs   
4      J A N U A R Y         10      Inherit the Viper   
..               ...        ...                    ...   
273  D E C E M B E R         25       We Can Be Heroes   
274  D E C E M B E R         25      News of the World   
275  D E C E M B E R         25  One Night in Miami...   
276  D E C E M B E R         25  Promising Young Woman   
277  D E C E M B E R         30      Pieces of a Woman   

                                    Production company  \
0    Screen Gems / Stage 6 Films / Ghost House Pict...   
1    20th Century Fox / TSG Entertainment / Chernin...   
2                   Paramount Pictures / Artists First   
3                                            IFC Films   
4    Lionsgate / Barry Films / Tycor International ...   
..                                                 ...   
273  Netflix / Troublemaker Studios / Double R Prod...   
274  Universal Pictures / Playtone / Perfect World ...   
275                                     Amazon Studios   
276          Focus Features / FilmNation Entertainment   
277                             Netflix / Bron Studios   

                                         Cast and crew   Ref.  
0    Nicolas Pesce (director/screenplay); Andrea Ri...    [2]  
1    William Eubank (director); Brian Duffield, Ada...    [3]  
2    Miguel Arteta (director); Sam Pitman, Adam Col...    [4]  
3    Jon Avnet (director/screenplay); Eric Nazarian...    NaN  
4    Anthony Jerjen (director); Andrew Crabtree (sc...    [5]  
..                                                 ...    ...  
273  Robert Rodriguez (director/screenplay); Priyan...  [246]  
274  Paul Greengrass (director/screenplay); Luke Da...  [247]  
275  Regina King (director); Kemp Powers (screenpla...  [248]  
276  Emerald Fennell (director/screenplay); Carey M...  [249]  
277  Kornél Mundruczó (director); Kata Wéber (scree...  [250]  

[278 rows x 6 columns]

In [23]:
df_2020 = df.loc[:,['Title','Cast and crew']].copy()

In [24]:
df_2020

Title                                      Cast and crew
0               The Grudge  Nicolas Pesce (director/screenplay); Andrea Ri...
1               Underwater  William Eubank (director); Brian Duffield, Ada...
2              Like a Boss  Miguel Arteta (director); Sam Pitman, Adam Col...
3            Three Christs  Jon Avnet (director/screenplay); Eric Nazarian...
4        Inherit the Viper  Anthony Jerjen (director); Andrew Crabtree (sc...
..                     ...                                                ...
273       We Can Be Heroes  Robert Rodriguez (director/screenplay); Priyan...
274      News of the World  Paul Greengrass (director/screenplay); Luke Da...
275  One Night in Miami...  Regina King (director); Kemp Powers (screenpla...
276  Promising Young Woman  Emerald Fennell (director/screenplay); Carey M...
277      Pieces of a Woman  Kornél Mundruczó (director); Kata Wéber (scree...

[278 rows x 2 columns]

In [30]:
!pip install tmdbv3api

In [31]:
from tmdbv3api import TMDb
import json
import requests
tmdb = TMDb()
tmdb.api_key = '3f9c4690b06b5114465b4cc0c2b03d8b'

In [34]:
from tmdbv3api import Movie
import requests
import numpy as np

tmdb_movie = Movie()

def get_genre(title):
    try:
        result = tmdb_movie.search(title)

        # Check if result is a list with at least one item
        if not result or not hasattr(result, '__getitem__'):
            return np.nan

        # Safely get the first result
        movie_id = getattr(result[0], 'id', None)
        if movie_id is None:
            return np.nan

        # Fetch genre info from TMDB
        url = f'https://api.themoviedb.org/3/movie/{movie_id}?api_key={tmdb.api_key}'
        response = requests.get(url)
        data_json = response.json()

        if 'genres' in data_json and data_json['genres']:
            genres = [g['name'] for g in data_json['genres']]
            return ", ".join(genres)
        else:
            return np.nan
    except Exception as e:
        print(f"Error for title '{title}':", e)
        return np.nan


In [35]:
df_2020['genres'] = df_2020['Title'].map(lambda x: get_genre(str(x)))

Error for title 'No films had been released on this date as resulting the closure of theaters caused by the COVID-19 pandemic': attribute name must be string, not 'int'
Error for title 'The SpongeBob Movie: Sponge on the Run (Canadian theatrical release)': attribute name must be string, not 'int'


In [36]:
df_2020

Title                                      Cast and crew  \
0               The Grudge  Nicolas Pesce (director/screenplay); Andrea Ri...   
1               Underwater  William Eubank (director); Brian Duffield, Ada...   
2              Like a Boss  Miguel Arteta (director); Sam Pitman, Adam Col...   
3            Three Christs  Jon Avnet (director/screenplay); Eric Nazarian...   
4        Inherit the Viper  Anthony Jerjen (director); Andrew Crabtree (sc...   
..                     ...                                                ...   
273       We Can Be Heroes  Robert Rodriguez (director/screenplay); Priyan...   
274      News of the World  Paul Greengrass (director/screenplay); Luke Da...   
275  One Night in Miami...  Regina King (director); Kemp Powers (screenpla...   
276  Promising Young Woman  Emerald Fennell (director/screenplay); Carey M...   
277      Pieces of a Woman  Kornél Mundruczó (director); Kata Wéber (scree...   

                              genres  
0          Horror, Mystery, Thriller  
1                              Drama  
2                             Comedy  
3                              Drama  
4             Crime, Thriller, Drama  
..                               ...  
273  Family, Action, Fantasy, Comedy  
274        Drama, Western, Adventure  
275                            Drama  
276           Thriller, Crime, Drama  
277                            Drama  

[278 rows x 3 columns]

In [39]:
def get_director(x):
    if " (director)" in x:
        return x.split(" (director)")[0]
    elif " (directors)" in x:
        return x.split(" (directors)")[0]
    else:
        return x.split(" (director/screenplay)")[0]

In [40]:
df_2020['director_name'] = df_2020['Cast and crew'].map(lambda x: get_director(str(x)))

In [41]:
def get_actor1(x):
    return ((x.split("screenplay); ")[-1]).split(", ")[0])

In [42]:
df_2020['actor_1_name'] = df_2020['Cast and crew'].map(lambda x: get_actor1(str(x)))

In [46]:
def get_actor2(x):
    if len((x.split("screenplay); ")[-1]).split(", ")) < 2:
        return np.nan
    else:
        return ((x.split("screenplay); ")[-1]).split(", ")[1])

In [47]:
df_2020['actor_2_name'] = df_2020['Cast and crew'].map(lambda x: get_actor2(str(x)))

In [49]:
def get_actor3(x):
    if len((x.split("screenplay); ")[-1]).split(", ")) < 3:
        return np.nan
    else:
        return ((x.split("screenplay); ")[-1]).split(", ")[2])

In [50]:
df_2020['actor_3_name'] = df_2020['Cast and crew'].map(lambda x: get_actor3(str(x)))

In [51]:
df_2020

Title                                      Cast and crew  \
0               The Grudge  Nicolas Pesce (director/screenplay); Andrea Ri...   
1               Underwater  William Eubank (director); Brian Duffield, Ada...   
2              Like a Boss  Miguel Arteta (director); Sam Pitman, Adam Col...   
3            Three Christs  Jon Avnet (director/screenplay); Eric Nazarian...   
4        Inherit the Viper  Anthony Jerjen (director); Andrew Crabtree (sc...   
..                     ...                                                ...   
273       We Can Be Heroes  Robert Rodriguez (director/screenplay); Priyan...   
274      News of the World  Paul Greengrass (director/screenplay); Luke Da...   
275  One Night in Miami...  Regina King (director); Kemp Powers (screenpla...   
276  Promising Young Woman  Emerald Fennell (director/screenplay); Carey M...   
277      Pieces of a Woman  Kornél Mundruczó (director); Kata Wéber (scree...   

                              genres     director_name           actor_1_name  \
0          Horror, Mystery, Thriller     Nicolas Pesce     Andrea Riseborough   
1                              Drama    William Eubank        Kristen Stewart   
2                             Comedy     Miguel Arteta        Tiffany Haddish   
3                              Drama         Jon Avnet           Richard Gere   
4             Crime, Thriller, Drama    Anthony Jerjen          Josh Hartnett   
..                               ...               ...                    ...   
273  Family, Action, Fantasy, Comedy  Robert Rodriguez  Priyanka Chopra Jonas   
274        Drama, Western, Adventure   Paul Greengrass              Tom Hanks   
275                            Drama       Regina King      Kingsley Ben-Adir   
276           Thriller, Crime, Drama   Emerald Fennell         Carey Mulligan   
277                            Drama  Kornél Mundruczó          Vanessa Kirby   

          actor_2_name     actor_3_name  
0        Demián Bichir         John Cho  
1       Vincent Cassel  Jessica Henwick  
2           Rose Byrne      Salma Hayek  
3       Peter Dinklage   Walton Goggins  
4    Margarita Levieva   Chandler Riggs  
..                 ...              ...  
273       Pedro Pascal    YaYa Gosselin  
274      Helena Zengel              NaN  
275          Eli Goree      Aldis Hodge  
276         Bo Burnham      Alison Brie  
277       Shia LaBeouf     Molly Parker  

[278 rows x 7 columns]

In [52]:
df_2020 = df_2020.rename(columns={'Title':'movie_title'})

In [53]:
new_df20 = df_2020.loc[:,['director_name','actor_1_name','actor_2_name','actor_3_name','genres','movie_title']]

In [54]:
new_df20

director_name           actor_1_name       actor_2_name  \
0       Nicolas Pesce     Andrea Riseborough      Demián Bichir   
1      William Eubank        Kristen Stewart     Vincent Cassel   
2       Miguel Arteta        Tiffany Haddish         Rose Byrne   
3           Jon Avnet           Richard Gere     Peter Dinklage   
4      Anthony Jerjen          Josh Hartnett  Margarita Levieva   
..                ...                    ...                ...   
273  Robert Rodriguez  Priyanka Chopra Jonas       Pedro Pascal   
274   Paul Greengrass              Tom Hanks      Helena Zengel   
275       Regina King      Kingsley Ben-Adir          Eli Goree   
276   Emerald Fennell         Carey Mulligan         Bo Burnham   
277  Kornél Mundruczó          Vanessa Kirby       Shia LaBeouf   

        actor_3_name                           genres            movie_title  
0           John Cho        Horror, Mystery, Thriller             The Grudge  
1    Jessica Henwick                            Drama             Underwater  
2        Salma Hayek                           Comedy            Like a Boss  
3     Walton Goggins                            Drama          Three Christs  
4     Chandler Riggs           Crime, Thriller, Drama      Inherit the Viper  
..               ...                              ...                    ...  
273    YaYa Gosselin  Family, Action, Fantasy, Comedy       We Can Be Heroes  
274              NaN        Drama, Western, Adventure      News of the World  
275      Aldis Hodge                            Drama  One Night in Miami...  
276      Alison Brie           Thriller, Crime, Drama  Promising Young Woman  
277     Molly Parker                            Drama      Pieces of a Woman  

[278 rows x 6 columns]

In [55]:
new_df20['comb'] = new_df20['actor_1_name'] + ' ' + new_df20['actor_2_name'] + ' '+ new_df20['actor_3_name'] + ' '+ new_df20['director_name'] +' ' + new_df20['genres']

In [56]:
new_df20.isna().sum()

director_name     0
actor_1_name      0
actor_2_name      5
actor_3_name     28
genres            2
movie_title       0
comb             29
dtype: int64

In [57]:
new_df20 = new_df20.dropna(how='any')

In [58]:
new_df20.isna().sum()

director_name    0
actor_1_name     0
actor_2_name     0
actor_3_name     0
genres           0
movie_title      0
comb             0
dtype: int64

In [59]:
new_df20['movie_title'] = new_df20['movie_title'].str.lower()

<ipython-input-59-2267385682>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df20['movie_title'] = new_df20['movie_title'].str.lower()


In [60]:
new_df20

director_name           actor_1_name       actor_2_name  \
0       Nicolas Pesce     Andrea Riseborough      Demián Bichir   
1      William Eubank        Kristen Stewart     Vincent Cassel   
2       Miguel Arteta        Tiffany Haddish         Rose Byrne   
3           Jon Avnet           Richard Gere     Peter Dinklage   
4      Anthony Jerjen          Josh Hartnett  Margarita Levieva   
..                ...                    ...                ...   
272       Pete Docter             Jamie Foxx           Tina Fey   
273  Robert Rodriguez  Priyanka Chopra Jonas       Pedro Pascal   
275       Regina King      Kingsley Ben-Adir          Eli Goree   
276   Emerald Fennell         Carey Mulligan         Bo Burnham   
277  Kornél Mundruczó          Vanessa Kirby       Shia LaBeouf   

        actor_3_name                              genres  \
0           John Cho           Horror, Mystery, Thriller   
1    Jessica Henwick                               Drama   
2        Salma Hayek                              Comedy   
3     Walton Goggins                               Drama   
4     Chandler Riggs              Crime, Thriller, Drama   
..               ...                                 ...   
272    Graham Norton  Animation, Family, Comedy, Fantasy   
273    YaYa Gosselin     Family, Action, Fantasy, Comedy   
275      Aldis Hodge                               Drama   
276      Alison Brie              Thriller, Crime, Drama   
277     Molly Parker                               Drama   

               movie_title                                               comb  
0               the grudge  Andrea Riseborough Demián Bichir John Cho Nico...  
1               underwater  Kristen Stewart Vincent Cassel Jessica Henwick...  
2              like a boss  Tiffany Haddish Rose Byrne Salma Hayek Miguel ...  
3            three christs  Richard Gere Peter Dinklage Walton Goggins Jon...  
4        inherit the viper  Josh Hartnett Margarita Levieva Chandler Riggs...  
..                     ...                                                ...  
272                   soul  Jamie Foxx Tina Fey Graham Norton Pete Docter ...  
273       we can be heroes  Priyanka Chopra Jonas Pedro Pascal YaYa Gossel...  
275  one night in miami...  Kingsley Ben-Adir Eli Goree Aldis Hodge Regina...  
276  promising young woman  Carey Mulligan Bo Burnham Alison Brie Emerald ...  
277      pieces of a woman  Vanessa Kirby Shia LaBeouf Molly Parker Kornél...  

[249 rows x 7 columns]

In [62]:
old_df = pd.read_csv('final_data.csv')

In [63]:
old_df

/usr/local/lib/python3.11/dist-packages/google/colab/_dataframe_summarizer.py:57: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  pd.to_datetime(column, errors="raise")


director_name       actor_1_name          actor_2_name  \
0              James Cameron        CCH Pounder      Joel David Moore   
1             Gore Verbinski        Johnny Depp         Orlando Bloom   
2                 Sam Mendes    Christoph Waltz          Rory Kinnear   
3          Christopher Nolan          Tom Hardy        Christian Bale   
4                Doug Walker        Doug Walker            Rob Walker   
...                      ...                ...                   ...   
5856  Nick Bruno, Troy Quane         Will Smith           Tom Holland   
5857            Greta Gerwig      Saoirse Ronan           Emma Watson   
5858              Sam Mendes      George MacKay  Dean-Charles Chapman   
5859   Destin Daniel Cretton  Michael B. Jordan            Jamie Foxx   
5860         Chinonye Chukwu      Alfre Woodard        Wendell Pierce   

              actor_3_name                                    genres  \
0                Wes Studi           Action Adventure Fantasy Sci-Fi   
1           Jack Davenport                  Action Adventure Fantasy   
2         Stephanie Sigman                 Action Adventure Thriller   
3     Joseph Gordon-Levitt                           Action Thriller   
4                  unknown                               Documentary   
...                    ...                                       ...   
5856         Rashida Jones  Animation Action Adventure Comedy Family   
5857         Florence Pugh                      Romance Drama Family   
5858           Mark Strong                War History Thriller Drama   
5859           Brie Larson                       Drama Crime History   
5860           Aldis Hodge                                     Drama   

                                            movie_title  \
0                                                avatar   
1              pirates of the caribbean: at world's end   
2                                               spectre   
3                                 the dark knight rises   
4     star wars: episode vii - the force awakens    ...   
...                                                 ...   
5856                                  spies in disguise   
5857                                       little women   
5858                                               1917   
5859                                         just mercy   
5860                                           clemency   

                                                   comb  
0     CCH Pounder Joel David Moore Wes Studi James C...  
1     Johnny Depp Orlando Bloom Jack Davenport Gore ...  
2     Christoph Waltz Rory Kinnear Stephanie Sigman ...  
3     Tom Hardy Christian Bale Joseph Gordon-Levitt ...  
4     Doug Walker Rob Walker unknown Doug Walker Doc...  
...                                                 ...  
5856  Will Smith Tom Holland Rashida Jones Nick Brun...  
5857  Saoirse Ronan Emma Watson Florence Pugh Greta ...  
5858  George MacKay Dean-Charles Chapman Mark Strong...  
5859  Michael B. Jordan Jamie Foxx Brie Larson Desti...  
5860  Alfre Woodard Wendell Pierce Aldis Hodge Chino...  

[5861 rows x 7 columns]

In [65]:
final_df = pd.concat([old_df, new_df20], ignore_index=True)

In [66]:
final_df

director_name           actor_1_name      actor_2_name  \
0         James Cameron            CCH Pounder  Joel David Moore   
1        Gore Verbinski            Johnny Depp     Orlando Bloom   
2            Sam Mendes        Christoph Waltz      Rory Kinnear   
3     Christopher Nolan              Tom Hardy    Christian Bale   
4           Doug Walker            Doug Walker        Rob Walker   
...                 ...                    ...               ...   
6105        Pete Docter             Jamie Foxx          Tina Fey   
6106   Robert Rodriguez  Priyanka Chopra Jonas      Pedro Pascal   
6107        Regina King      Kingsley Ben-Adir         Eli Goree   
6108    Emerald Fennell         Carey Mulligan        Bo Burnham   
6109   Kornél Mundruczó          Vanessa Kirby      Shia LaBeouf   

              actor_3_name                              genres  \
0                Wes Studi     Action Adventure Fantasy Sci-Fi   
1           Jack Davenport            Action Adventure Fantasy   
2         Stephanie Sigman           Action Adventure Thriller   
3     Joseph Gordon-Levitt                     Action Thriller   
4                  unknown                         Documentary   
...                    ...                                 ...   
6105         Graham Norton  Animation, Family, Comedy, Fantasy   
6106         YaYa Gosselin     Family, Action, Fantasy, Comedy   
6107           Aldis Hodge                               Drama   
6108           Alison Brie              Thriller, Crime, Drama   
6109          Molly Parker                               Drama   

                                            movie_title  \
0                                                avatar   
1              pirates of the caribbean: at world's end   
2                                               spectre   
3                                 the dark knight rises   
4     star wars: episode vii - the force awakens    ...   
...                                                 ...   
6105                                               soul   
6106                                   we can be heroes   
6107                              one night in miami...   
6108                              promising young woman   
6109                                  pieces of a woman   

                                                   comb  
0     CCH Pounder Joel David Moore Wes Studi James C...  
1     Johnny Depp Orlando Bloom Jack Davenport Gore ...  
2     Christoph Waltz Rory Kinnear Stephanie Sigman ...  
3     Tom Hardy Christian Bale Joseph Gordon-Levitt ...  
4     Doug Walker Rob Walker unknown Doug Walker Doc...  
...                                                 ...  
6105  Jamie Foxx Tina Fey Graham Norton Pete Docter ...  
6106  Priyanka Chopra Jonas Pedro Pascal YaYa Gossel...  
6107  Kingsley Ben-Adir Eli Goree Aldis Hodge Regina...  
6108  Carey Mulligan Bo Burnham Alison Brie Emerald ...  
6109  Vanessa Kirby Shia LaBeouf Molly Parker Kornél...  

[6110 rows x 7 columns]

In [67]:
final_df.to_csv('main_data.csv',index=False)